In [ ]:
import glob
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
import seaborn as sn
from methods import *
from cnn import *

In [ ]:
food_imgs = []
for path in glob.glob("./FOOD/*.JPG"):
    if path == './FOOD\\mix002T(2).JPG':
        continue
    elif path == './FOOD\\mix005S(4).JPG':
        continue
    else: food_imgs.append(path)
        
annotations = glob.glob("./Annotations/*.xml")

In [ ]:
food_bbox, coin_bbox = get_bbox(annotations, [], [])
info = create_df(food_bbox, coin_bbox)
food_df = get_food_df()

In [ ]:
xtr, xtest, y_train, y_test = train_test_split(info, info.label, test_size=0.3, random_state=42)
xtest, xval, y_test, y_val = train_test_split(xtest, y_test, test_size=0.5, random_state=42)

indices = list(xtest.index)

X_train = image_data([], xtr)
X_val = image_data([], xval)
X_test = image_data([], xtest)

In [ ]:
history = model.fit(X_train, y_train, batch_size=32, epochs=50, validation_data=(X_val, y_val), shuffle=True)

#### Dataset to test volume estimation

In [ ]:
real_mean_volume = [densities[c].mean(axis=0)['volume(mm^3)'] for c in classes]

In [ ]:
test_df = info.groupby('label', as_index=False, group_keys=False).apply(lambda s: s.sample(1))
# get 3 random samples from each class

In [ ]:
test_data = image_data([], test_df)

In [ ]:
test_idxs = list(test_df.index)

In [ ]:
test_yhat = model.predict_classes(test_data)

In [ ]:
test_vol_pred = []

for i in range(len(test_yhat)):
    idx = test_idxs[i]
    label = test_yhat[i]
    volume = get_volume(idx, label, info)
    calorie = get_calorie(volume, food_df.iloc[label]['density'], food_df.iloc[label]['calorie'])
    # print("Calorie : ", calorie)
    test_vol_pred.append(volume)

In [ ]:
MSE = np.square(np.subtract(real_mean_volume, test_vol_pred)).mean()

MSE